In [1]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import expon
from scipy.integrate import quad
import matplotlib.pyplot as plt
from scipy.stats import norm

##Expected shortfall

 Derive a formula for the expected shortfall if gains are normally distributed $N(\mu, \sigma^{2})$.

Normal Distribution:
\begin{equation}
f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}
\end{equation}

\begin{equation}
F(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\int_{-\infty}^x e^{-\frac{(t-\mu)^2}{2\sigma^2}} dt
\end{equation}

$$E[X] = \mu, Var[X] = \sigma^2$$

Expected Shortfall:\begin{align*}
ES_{\alpha} &= W_0-E[W|W<W_0-VAR]\\
& = W_0-\frac{\int_{-\infty}^{W_0-VAR}Wf(W)dW}{\int_{-\infty}^{W_0-VAR}f(W)dW}\\
\end{align*}

Derive the formula: Let \begin{align*}
y = \frac{x-\mu}{\sigma}
\end{align*}
We can get \begin{align*}
dy = \frac{dx}{\sigma}, x = {\sigma}y + \mu, dx = {\sigma}dy
\end{align*}
Then
$$
\begin{aligned}
ES_{\alpha} &=\frac{1}{\sqrt{2 \pi \sigma^2}} \int_{-\infty}^{\mu-VaR_\alpha} x \cdot \exp \left(-\frac{(x-\mu)^2}{2 \sigma^2}\right) d x \\
= & \frac{1}{\sqrt{2 \pi \sigma^2}} \int(\sigma y+\mu) \cdot \exp \left(-\frac{(\sigma y+\mu-\mu)^2}{2 \sigma^2}\right) \sigma d y \\
= & \frac{1}{\sqrt{2 \pi \sigma^2}} \int(\sigma y+\mu) \exp \left(-\frac{y^2}{2}\right) \sigma d y \\
= & \frac{1}{\sqrt{2 \pi \sigma^2}} \int \sigma^2 y \exp \left(-\frac{y^2}{2}\right) d y+\frac{1}{\sqrt{2 \pi \sigma^2}} \int \sigma \mu \exp \left(-\frac{y^2}{2}\right) d y \\
= & \mu \int_{-\infty}^{y_0-V a R_{(0,1)}} \frac{1}{\sqrt{2 \pi}} \exp \left(-\frac{y^2}{2}\right) d y+\sigma \int_{-\infty}^{y_0-V a R_{(0,1)}} \frac{1}{\sqrt{2 \pi}} y \exp \left(-\frac{y^2}{2}\right) d y
\end{aligned}
$$
Since
$$\begin{aligned}
y_0 = 0, VaR_\alpha = -z(1-\alpha)=-z(c))
\end{aligned}$$
We continue deriving the above formula, and we can abtain
$$
\begin{aligned}
= & \mu \cdot(1-c)+\left.\sigma \cdot \frac{-1}{\sqrt{2 \pi}} \cdot e^{\frac{-y^2}{2}}\right|_{\infty} ^{z(c)} \\
= & \mu \cdot(1-c)+\sigma \cdot \frac{-1}{\sqrt{2 \pi}} e^{-z(c)^2 / 2}
\end{aligned}
$$
Finally, we can get
$$
E S=y_0-\frac{\mu \cdot(1-c)+\sigma \cdot \frac{-1}{\sqrt{2 \pi}} e^{-z(c)^2 / 2}}{1-c}=-\mu+\sigma \cdot \frac{1}{\sqrt{2 \pi}(1-c)} e^{-z(c)^2 / 2}
$$

The expected shortfall can also be defined as the average of the VaR for all confidence level above c: $ES = \frac{1}{1-c}\int_{c}^{1} VaR_{\alpha}d\alpha$ Prove that this definition is equivalent to the one we have seen in class. Hint: You can use integration by part and a change of variable.

From the definition of VaR and ES, we get
\begin{aligned}
E S & =\frac{1}{1-c} \int_c^1 V a R_\alpha d \alpha \\
& =\frac{1}{1-c} \int_c^1 W_0-F^{-1}(1-\alpha) d \alpha \\
& =W_0-\frac{1}{1-c} \int_c^1 F^{-1}(1-\alpha) d \alpha
\end{aligned}
Cause$$
\begin{aligned}
W=F^{-1}(1-\alpha)
\end{aligned}
$$
We have
$$
d W=-\frac{1}{F^{\prime}\left(F^{-1}(1-\alpha)\right)} d \alpha=-\frac{1}{f\left(F^{-1}(1-\alpha)\right)} d \alpha
$$
Since
$$
d \alpha=-f\left(F^{-1}(1-\alpha)\right) d W=-f(W) d W
$$
and
$$F^{-1}(1-1)=-\infty, F^{-1}(1-c)=W_0-V a R$$
We have
$$
\begin{aligned}
& W_0-\frac{1}{1-c} \int_c^1 F^{-1}(1-\alpha) d \alpha \\
= & W_0-\frac{1}{1-c} \int_{W_0-V a R}^{-\infty}-W f(W) d W \\
= & W_0-\frac{1}{1-c} \int_{-\infty}^{W_0-V a R} W f(W) d W
\end{aligned}
$$

 Decomposing the VaR of a portfolio

In [ ]:
# This function creates N draws from some three random variables
import numpy as np

def draw_returns(N):

    # coin flips
    normal_year = np.random.binomial(1, 0.9, N)

    # draw for normal years
    mu = np.array([0.05, 0.05, 0.05])
    Sigma = np.array([[0.09, 0.012, 0.021], [0.012, 0.16, 0.028], [0.021, 0.028, 0.49]])
    normal_ret = np.random.multivariate_normal(mu, Sigma, N)

    # draws for special years
    mu = np.array([-0.1, -0.1, -0.1])
    Sigma = np.array([[0.36, 0.24, 0.42], [0.24, 0.64, 0.56], [0.42, 0.56, 1.96]])
    special_ret = np.random.multivariate_normal(mu, Sigma, N)

    # combine
    ret = normal_ret
    for i in range(N):
        if normal_year[i] == 0:
            ret[i,:] = special_ret[i,:]

    return(ret)

 Assume you have a portfolio of 2m dollars in asset A, 6m dollars in asset B and 2m dollars in asset C. What is the VaR of your portfolio? What is the CVaR and DVaR for each of the assets? Check that the sum of CVaRs coincides with VaR. Which asset is responsible for the most risk of the portfolio?

In [ ]:
def VaR(value_asset):
    np.random.seed(123)
    # Get the returns of the portfolio
    N = 10000
    ret_draw = draw_returns(N)
    ret_portfolio = np.dot(ret_draw, value_asset.T)

    # Get the return at c=99%
    VaR = -np.quantile(ret_portfolio, 0.01)

    return VaR

value_asset = np.array([2000000, 6000000, 2000000]).reshape(1, -1)
VaR_portfolio = VaR(value_asset)
print("The 99% VaR of the portfolio is", VaR_portfolio, "dollars.")

The 99% VaR of the portfolio is 10759232.431098873 dollars.
